# Lab D.3: Deep Q-Networks (DQN)

**Module:** D - Reinforcement Learning (Optional)
**Time:** 2-2.5 hours
**Difficulty:** ⭐⭐⭐⭐☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why neural networks are needed for large state spaces
- [ ] Implement a Q-network with PyTorch
- [ ] Build an experience replay buffer
- [ ] Implement target networks for training stability
- [ ] Train DQN to solve CartPole (achieving >450 reward)
- [ ] Ablate key components to understand their importance

---

## 📚 Prerequisites

- Completed: Lab D.2 (Q-Learning)
- Knowledge of: PyTorch basics (Module 2.1)
- Understanding of: Neural networks, backpropagation

---

## 🌍 Real-World Context

**The Problem with Tabular Q-Learning:**

In Lab D.2, we stored Q-values in a table with one entry per (state, action) pair. This works for small, discrete state spaces.

But consider:
- **Atari**: 210×160 pixels, 128 colors → astronomical number of states
- **Robotics**: Continuous sensor readings → infinite state space
- **LLMs**: Context = all previous tokens → effectively infinite

**The DQN breakthrough (2013):** Use a neural network to **approximate** $Q(s, a)$!

DeepMind's DQN was the first to achieve human-level performance on Atari games, launching the deep reinforcement learning revolution.

---

## 🧒 ELI5: What is a Deep Q-Network?

> **Imagine you have a Q-table so big you can't possibly fill it in.** 📊
>
> Instead of memorizing every single combination (impossible!), you learn **patterns**:
> - "States that look like THIS tend to have high value"
> - "When the enemy is close, dodging is usually good"
>
> A neural network is a "pattern recognizer" that can generalize:
> - It sees a few million examples
> - It learns to predict Q-values for states it's never seen!
>
> **The magic:** Instead of $Q[s][a]$ = lookup in a table, we have $Q(s, a; \theta)$ = neural network with parameters $\theta$.
>
> **In AI terms:** DQN uses a neural network as a function approximator for the Q-function. The network takes a state as input and outputs Q-values for all possible actions.

---

## Part 1: Setup and CartPole Environment

**CartPole**: Balance a pole on a cart by moving left or right.

- **State**: 4 continuous values (cart position, cart velocity, pole angle, pole velocity)
- **Actions**: 2 (push left, push right)
- **Reward**: +1 for each timestep the pole is balanced
- **Goal**: Score 500 (episode lasts 500 steps) or average >475 over 100 episodes

In [ ]:
# Setup - run this first!
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
from collections import deque
import random
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Gymnasium
try:
    import gymnasium as gym
except ImportError:
    !pip install gymnasium -q
    import gymnasium as gym

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Device selection - prioritize GPU on DGX Spark
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("🚀 Module D.3: Deep Q-Networks")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Create CartPole environment
env = gym.make("CartPole-v1")

print("🎮 CartPole-v1 Environment")
print(f"   State space: {env.observation_space}")
print(f"   State shape: {env.observation_space.shape}")
print(f"   Action space: {env.action_space}")
print(f"   Max episode steps: 500")
print()
print("   State components:")
print("   [0] Cart Position   (-4.8 to 4.8)")
print("   [1] Cart Velocity   (-Inf to Inf)")
print("   [2] Pole Angle      (-0.42 rad to 0.42 rad)")
print("   [3] Pole Velocity   (-Inf to Inf)")

In [ ]:
# Let's see what random play looks like

def test_random_agent(env, n_episodes: int = 10):
    """See how random actions perform."""
    rewards = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        
        for step in range(500):
            action = env.action_space.sample()  # Random action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            
            if done:
                break
        
        rewards.append(total_reward)
    
    print(f"Random Agent Performance ({n_episodes} episodes):")
    print(f"   Mean reward: {np.mean(rewards):.1f}")
    print(f"   Max reward:  {np.max(rewards):.1f}")
    print(f"   (Goal: >475 average, ideally 500)")
    
    return rewards

random_rewards = test_random_agent(env)

---

## Part 2: Building the Q-Network

Our neural network takes a **state** as input and outputs **Q-values for each action**:

```
State [4 dims] → Hidden [64] → Hidden [64] → Q-values [2]
```

In [ ]:
class QNetwork(nn.Module):
    """
    Neural network for Q-value approximation.
    
    Takes state as input, outputs Q(s, a) for all actions.
    
    Architecture:
        Input (state_dim) → FC(64) → ReLU → FC(64) → ReLU → Output (action_dim)
    """
    
    def __init__(self, state_dim: int, action_dim: int, hidden_dim: int = 64):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
        
        # Initialize weights
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.orthogonal_(layer.weight)
                nn.init.zeros_(layer.bias)
    
    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            state: Shape (batch_size, state_dim) or (state_dim,)
        
        Returns:
            Q-values for each action, shape (batch_size, action_dim)
        """
        return self.network(state)

# Create a test network
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

test_net = QNetwork(state_dim, action_dim).to(device)
print(f"📐 Q-Network Architecture:")
print(test_net)
print(f"\n📊 Parameters: {sum(p.numel() for p in test_net.parameters()):,}")

In [ ]:
# Test the network
state, _ = env.reset()
state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)  # Add batch dim

with torch.no_grad():
    q_values = test_net(state_tensor)

print(f"Input state: {state}")
print(f"Q-values: {q_values.cpu().numpy()[0]}")
print(f"Best action: {q_values.argmax().item()} ({'Left' if q_values.argmax().item() == 0 else 'Right'})")

---

## Part 3: Experience Replay Buffer

> 🧒 **ELI5**: Imagine you're studying for an exam. Would you only review the last thing you learned? No! You'd go back and re-study earlier material too.
>
> Experience replay is like that—we store past experiences and randomly sample them for learning. This:
> 1. **Breaks correlations**: Sequential experiences are highly correlated (bad for learning!)
> 2. **Reuses data**: Each experience can be learned from multiple times
> 3. **Smooths learning**: Don't forget old lessons when learning new ones

In [ ]:
class ReplayBuffer:
    """
    Experience Replay Buffer for DQN.
    
    Stores (state, action, reward, next_state, done) tuples.
    Randomly samples batches for training.
    """
    
    def __init__(self, capacity: int = 100000):
        """
        Initialize the replay buffer.
        
        Args:
            capacity: Maximum number of experiences to store
        """
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
    
    def push(self, state: np.ndarray, action: int, reward: float,
             next_state: np.ndarray, done: bool):
        """
        Add an experience to the buffer.
        """
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size: int) -> Tuple[torch.Tensor, ...]:
        """
        Randomly sample a batch of experiences.
        
        Returns tensors ready for training.
        """
        batch = random.sample(self.buffer, batch_size)
        
        states, actions, rewards, next_states, dones = zip(*batch)
        
        return (
            torch.FloatTensor(np.array(states)).to(device),
            torch.LongTensor(actions).to(device),
            torch.FloatTensor(rewards).to(device),
            torch.FloatTensor(np.array(next_states)).to(device),
            torch.FloatTensor(dones).to(device)
        )
    
    def __len__(self) -> int:
        return len(self.buffer)

# Test the buffer
buffer = ReplayBuffer(capacity=10000)

# Add some fake experiences
for i in range(100):
    fake_state = np.random.randn(4)
    fake_action = random.randint(0, 1)
    fake_reward = random.random()
    fake_next_state = np.random.randn(4)
    fake_done = random.random() < 0.1
    buffer.push(fake_state, fake_action, fake_reward, fake_next_state, fake_done)

print(f"Buffer size: {len(buffer)}")

# Sample a batch
states, actions, rewards, next_states, dones = buffer.sample(32)
print(f"Sampled batch shapes:")
print(f"   States: {states.shape}")
print(f"   Actions: {actions.shape}")
print(f"   Rewards: {rewards.shape}")

---

## Part 4: DQN Agent with Target Network

### The Stability Problem

In Q-learning, we update towards: $y = r + \gamma \max_{a'} Q(s', a')$

But $Q$ is the network we're training! This creates a **moving target**.

> 🧒 **ELI5**: Imagine trying to hit a bullseye, but every time you throw a dart, someone moves the target. Hard to improve!

**Solution: Target Network**
- Keep a separate copy of the network for computing targets
- Update the target network slowly (every N steps)
- This gives a "stable" target to learn from

In [ ]:
class DQNAgent:
    """
    Deep Q-Network Agent.
    
    Features:
    - Neural network for Q-value approximation
    - Experience replay buffer
    - Target network for stable training
    - Epsilon-greedy exploration
    """
    
    def __init__(self,
                 state_dim: int,
                 action_dim: int,
                 hidden_dim: int = 64,
                 lr: float = 1e-3,
                 gamma: float = 0.99,
                 buffer_size: int = 100000,
                 batch_size: int = 64,
                 target_update_freq: int = 100,
                 epsilon_start: float = 1.0,
                 epsilon_end: float = 0.01,
                 epsilon_decay_steps: int = 10000):
        """
        Initialize the DQN agent.
        """
        self.action_dim = action_dim
        self.gamma = gamma
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        
        # Exploration parameters
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = (epsilon_start - epsilon_end) / epsilon_decay_steps
        
        # Q-Network (the one we train)
        self.q_network = QNetwork(state_dim, action_dim, hidden_dim).to(device)
        
        # Target Network (for stable targets)
        self.target_network = QNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()  # Target never trains directly
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        
        # Replay buffer
        self.buffer = ReplayBuffer(buffer_size)
        
        # Tracking
        self.train_steps = 0
        self.losses = []
        
        print(f"🤖 DQN Agent Initialized")
        print(f"   Q-Network parameters: {sum(p.numel() for p in self.q_network.parameters()):,}")
        print(f"   Target update frequency: every {target_update_freq} steps")
        print(f"   Epsilon decay: {epsilon_start} → {epsilon_end} over {epsilon_decay_steps} steps")
    
    def select_action(self, state: np.ndarray, training: bool = True) -> int:
        """
        Select an action using epsilon-greedy policy.
        """
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_values = self.q_network(state_tensor)
            return q_values.argmax().item()
    
    def store_experience(self, state, action, reward, next_state, done):
        """Store an experience in the replay buffer."""
        self.buffer.push(state, action, reward, next_state, done)
    
    def train_step(self) -> Optional[float]:
        """
        Perform one training step on a batch from the replay buffer.
        
        Returns the loss (or None if buffer too small).
        """
        if len(self.buffer) < self.batch_size:
            return None
        
        # Sample batch
        states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        
        # Current Q-values: Q(s, a) for the actions we took
        current_q = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Target Q-values: r + γ * max_a' Q_target(s', a')
        with torch.no_grad():
            next_q = self.target_network(next_states).max(dim=1)[0]
            target_q = rewards + self.gamma * next_q * (1 - dones)
        
        # Loss: MSE between current and target Q-values
        loss = F.mse_loss(current_q, target_q)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        # Track
        self.train_steps += 1
        self.losses.append(loss.item())
        
        # Update target network periodically
        if self.train_steps % self.target_update_freq == 0:
            self.update_target_network()
        
        # Decay epsilon
        self.epsilon = max(self.epsilon_end, self.epsilon - self.epsilon_decay)
        
        return loss.item()
    
    def update_target_network(self):
        """Copy Q-network weights to target network."""
        self.target_network.load_state_dict(self.q_network.state_dict())

# Create our agent!
agent = DQNAgent(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.n,
    hidden_dim=64,
    lr=1e-3,
    gamma=0.99,
    buffer_size=50000,
    batch_size=64,
    target_update_freq=100,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay_steps=5000
)

---

## Part 5: Training DQN

In [ ]:
def train_dqn(env, agent, n_episodes: int = 500, 
              max_steps: int = 500,
              eval_freq: int = 50) -> Dict:
    """
    Train the DQN agent.
    
    Returns training statistics.
    """
    episode_rewards = []
    episode_lengths = []
    eval_rewards = []
    
    best_eval_reward = 0
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        
        for step in range(max_steps):
            # Select and take action
            action = agent.select_action(state, training=True)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Store experience
            agent.store_experience(state, action, reward, next_state, done)
            
            # Train
            agent.train_step()
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        episode_rewards.append(total_reward)
        episode_lengths.append(step + 1)
        
        # Periodic evaluation
        if (episode + 1) % eval_freq == 0:
            eval_reward = evaluate_agent(env, agent)
            eval_rewards.append((episode, eval_reward))
            
            if eval_reward > best_eval_reward:
                best_eval_reward = eval_reward
            
            # Print progress
            recent_reward = np.mean(episode_rewards[-50:])
            print(f"Episode {episode + 1:4d} | "
                  f"Recent Avg: {recent_reward:6.1f} | "
                  f"Eval: {eval_reward:6.1f} | "
                  f"ε: {agent.epsilon:.3f} | "
                  f"Buffer: {len(agent.buffer):,}")
    
    return {
        'episode_rewards': episode_rewards,
        'episode_lengths': episode_lengths,
        'eval_rewards': eval_rewards,
        'losses': agent.losses,
        'best_eval': best_eval_reward
    }


def evaluate_agent(env, agent, n_episodes: int = 10) -> float:
    """
    Evaluate the agent's policy (no exploration).
    """
    total_rewards = []
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        episode_reward = 0
        
        for step in range(500):
            action = agent.select_action(state, training=False)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            state = next_state
            
            if done:
                break
        
        total_rewards.append(episode_reward)
    
    return np.mean(total_rewards)

In [ ]:
# Train the agent!
print("\n🏋️ Training DQN on CartPole-v1...\n")

stats = train_dqn(env, agent, n_episodes=400, eval_freq=50)

print(f"\n✅ Training Complete!")
print(f"   Best evaluation reward: {stats['best_eval']:.1f}")

In [ ]:
# Visualize training progress

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Episode rewards
ax = axes[0, 0]
ax.plot(stats['episode_rewards'], alpha=0.3, label='Raw')
window = 50
smoothed = np.convolve(stats['episode_rewards'], np.ones(window)/window, mode='valid')
ax.plot(range(window-1, len(stats['episode_rewards'])), smoothed, label=f'{window}-episode avg')
ax.axhline(y=475, color='g', linestyle='--', label='Target (475)')
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('Training Rewards')
ax.legend()
ax.grid(True, alpha=0.3)

# Evaluation rewards
ax = axes[0, 1]
if stats['eval_rewards']:
    eval_eps, eval_rews = zip(*stats['eval_rewards'])
    ax.plot(eval_eps, eval_rews, 'o-', markersize=8)
    ax.axhline(y=450, color='g', linestyle='--', label='Goal (450)')
ax.set_xlabel('Episode')
ax.set_ylabel('Evaluation Reward')
ax.set_title('Evaluation Performance')
ax.legend()
ax.grid(True, alpha=0.3)

# Loss
ax = axes[1, 0]
if stats['losses']:
    window = 100
    if len(stats['losses']) > window:
        smoothed_loss = np.convolve(stats['losses'], np.ones(window)/window, mode='valid')
        ax.plot(smoothed_loss)
    else:
        ax.plot(stats['losses'])
ax.set_xlabel('Training Step')
ax.set_ylabel('Loss (smoothed)')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.3)

# Episode lengths
ax = axes[1, 1]
ax.plot(stats['episode_lengths'], alpha=0.3)
window = 50
smoothed_lens = np.convolve(stats['episode_lengths'], np.ones(window)/window, mode='valid')
ax.plot(range(window-1, len(stats['episode_lengths'])), smoothed_lens)
ax.axhline(y=500, color='g', linestyle='--', label='Max (500)')
ax.set_xlabel('Episode')
ax.set_ylabel('Episode Length')
ax.set_title('Episode Length (longer = better balancing)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Final evaluation
final_reward = evaluate_agent(env, agent, n_episodes=100)
print(f"\n📊 Final Evaluation (100 episodes):")
print(f"   Mean Reward: {final_reward:.1f}")

if final_reward >= 450:
    print("\n🎉 Achieved >450 average reward! Lab objective complete!")
else:
    print("\n⚠️ Try training longer or adjusting hyperparameters.")

---

## Part 6: Ablation Study

Let's see how important each component is by removing them one at a time.

In [ ]:
class DQNAgentNoReplay(DQNAgent):
    """
    DQN without experience replay - trains on most recent experience only.
    """
    
    def train_step(self) -> Optional[float]:
        if len(self.buffer) == 0:
            return None
        
        # Use only the most recent experience (no replay!)
        state, action, reward, next_state, done = self.buffer.buffer[-1]
        
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        action = torch.LongTensor([action]).to(device)
        reward = torch.FloatTensor([reward]).to(device)
        next_state = torch.FloatTensor(next_state).unsqueeze(0).to(device)
        done = torch.FloatTensor([done]).to(device)
        
        # Standard DQN update (same as before)
        current_q = self.q_network(state).gather(1, action.unsqueeze(1)).squeeze(1)
        
        with torch.no_grad():
            next_q = self.target_network(next_state).max(dim=1)[0]
            target_q = reward + self.gamma * next_q * (1 - done)
        
        loss = F.mse_loss(current_q, target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.train_steps += 1
        self.losses.append(loss.item())
        
        if self.train_steps % self.target_update_freq == 0:
            self.update_target_network()
        
        self.epsilon = max(self.epsilon_end, self.epsilon - self.epsilon_decay)
        
        return loss.item()


class DQNAgentNoTarget(DQNAgent):
    """
    DQN without target network - uses Q-network for targets.
    """
    
    def train_step(self) -> Optional[float]:
        if len(self.buffer) < self.batch_size:
            return None
        
        states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        
        current_q = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Use Q-network instead of target network!
        with torch.no_grad():
            next_q = self.q_network(next_states).max(dim=1)[0]  # Same network!
            target_q = rewards + self.gamma * next_q * (1 - dones)
        
        loss = F.mse_loss(current_q, target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        self.train_steps += 1
        self.losses.append(loss.item())
        
        self.epsilon = max(self.epsilon_end, self.epsilon - self.epsilon_decay)
        
        return loss.item()

In [ ]:
def run_ablation_study(env, n_episodes: int = 300, n_trials: int = 2):
    """
    Compare full DQN vs ablated versions.
    """
    results = {}
    
    configs = [
        ("Full DQN", DQNAgent),
        ("No Replay", DQNAgentNoReplay),
        ("No Target Net", DQNAgentNoTarget),
    ]
    
    for name, agent_class in configs:
        print(f"\n{'='*50}")
        print(f"Testing: {name}")
        print(f"{'='*50}")
        
        trial_rewards = []
        
        for trial in range(n_trials):
            print(f"\nTrial {trial + 1}/{n_trials}")
            
            agent = agent_class(
                state_dim=env.observation_space.shape[0],
                action_dim=env.action_space.n,
                hidden_dim=64,
                lr=1e-3,
                gamma=0.99,
                epsilon_decay_steps=5000
            )
            
            stats = train_dqn(env, agent, n_episodes=n_episodes, eval_freq=100)
            
            final_reward = evaluate_agent(env, agent, n_episodes=50)
            trial_rewards.append(final_reward)
            print(f"   Trial {trial + 1} final reward: {final_reward:.1f}")
        
        results[name] = {
            'mean': np.mean(trial_rewards),
            'std': np.std(trial_rewards),
            'trials': trial_rewards
        }
    
    return results

# Run ablation (this takes a few minutes)
print("\n🔬 Running Ablation Study...")
ablation_results = run_ablation_study(env, n_episodes=200, n_trials=2)

In [ ]:
# Visualize ablation results

plt.figure(figsize=(10, 6))

names = list(ablation_results.keys())
means = [ablation_results[n]['mean'] for n in names]
stds = [ablation_results[n]['std'] for n in names]

bars = plt.bar(range(len(names)), means, yerr=stds, capsize=10, alpha=0.7,
               color=['green', 'orange', 'red'])
plt.xticks(range(len(names)), names, fontsize=12)
plt.ylabel('Average Reward (100 episodes)', fontsize=12)
plt.title('DQN Ablation Study: Importance of Each Component', fontsize=14)
plt.axhline(y=450, color='green', linestyle='--', label='Goal (450)')
plt.legend()

# Add value labels
for bar, mean, std in zip(bars, means, stds):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 10,
             f'{mean:.0f}', ha='center', fontsize=12, fontweight='bold')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n📊 Ablation Results:")
for name in names:
    r = ablation_results[name]
    print(f"   {name:15s}: {r['mean']:6.1f} ± {r['std']:5.1f}")

### Key Insights from Ablation

1. **Experience Replay** is crucial:
   - Without it, learning is highly unstable
   - Sequential experiences are correlated, breaking i.i.d. assumption

2. **Target Network** stabilizes training:
   - Without it, the "moving target" problem makes learning erratic
   - Loss may look low but policy doesn't improve

3. **Both components work together**:
   - Replay provides diverse, decorrelated samples
   - Target network provides stable learning signal

---

## ⚠️ Common Mistakes

### Mistake 1: Not Detaching Target

```python
# ❌ Gradients flow through target (unstable!)
next_q = self.target_network(next_states).max(dim=1)[0]
target_q = rewards + self.gamma * next_q * (1 - dones)
loss.backward()  # Target affects Q-network gradients!

# ✅ Detach target from gradient graph
with torch.no_grad():
    next_q = self.target_network(next_states).max(dim=1)[0]
    target_q = rewards + self.gamma * next_q * (1 - dones)
```

### Mistake 2: Gathering Wrong Dimension

```python
# ❌ Wrong: actions shape mismatch
q_values = self.q_network(states).gather(1, actions)  # Error!

# ✅ Actions need to be 2D for gather
q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
```

### Mistake 3: Terminal State Value

```python
# ❌ Including future value for terminal states
target_q = rewards + self.gamma * next_q  # Wrong for done=True!

# ✅ Zero future value when done
target_q = rewards + self.gamma * next_q * (1 - dones)
```

---

## 🎉 Checkpoint

You've learned:
- ✅ **Function Approximation**: Using neural networks for large state spaces
- ✅ **Q-Network**: Network that maps states to action values
- ✅ **Experience Replay**: Breaking correlations by random sampling
- ✅ **Target Networks**: Stable targets for training
- ✅ **DQN Algorithm**: Combining all components for stable deep RL

---

## 🚀 Challenge: Double DQN

Standard DQN overestimates Q-values because it uses the same network to select AND evaluate actions.

**Double DQN** decouples selection and evaluation:
- Use Q-network to **select** the best action
- Use target network to **evaluate** that action's value

In [ ]:
# Challenge: Implement Double DQN

class DoubleDQNAgent(DQNAgent):
    """
    Double DQN to reduce overestimation bias.
    """
    
    def train_step(self) -> Optional[float]:
        if len(self.buffer) < self.batch_size:
            return None
        
        states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        
        # Current Q-values
        current_q = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Double DQN target
        with torch.no_grad():
            # Select best action using Q-network
            best_actions = self.q_network(next_states).argmax(dim=1)
            
            # Evaluate that action using target network
            next_q = self.target_network(next_states).gather(1, best_actions.unsqueeze(1)).squeeze(1)
            
            target_q = rewards + self.gamma * next_q * (1 - dones)
        
        loss = F.mse_loss(current_q, target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        self.train_steps += 1
        self.losses.append(loss.item())
        
        if self.train_steps % self.target_update_freq == 0:
            self.update_target_network()
        
        self.epsilon = max(self.epsilon_end, self.epsilon - self.epsilon_decay)
        
        return loss.item()

# Test Double DQN
print("\n🏋️ Training Double DQN...")

double_dqn_agent = DoubleDQNAgent(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.n,
)

double_stats = train_dqn(env, double_dqn_agent, n_episodes=300, eval_freq=100)

final_reward = evaluate_agent(env, double_dqn_agent, n_episodes=100)
print(f"\n📊 Double DQN Final Reward: {final_reward:.1f}")

---

## 🔗 Connection to LLMs and RLHF

DQN taught us key principles that apply to LLM training:

| DQN Concept | RLHF Equivalent |
|-------------|------------------|
| Q-Network | The language model being fine-tuned |
| Target Network | Reference model (KL constraint) |
| Experience Replay | Training on batches of (prompt, response) pairs |
| Epsilon-greedy | Temperature in sampling |

However, LLMs use **policy gradient** methods (PPO) instead of value-based methods (DQN) because:
1. Text generation has huge action space (vocabulary size)
2. Policy methods directly optimize the generation distribution

We'll learn PPO in the next lab!

---

## 📖 Further Reading

- [DQN Paper (2013)](https://arxiv.org/abs/1312.5602) - Original breakthrough
- [Double DQN](https://arxiv.org/abs/1509.06461) - Reduces overestimation
- [Dueling DQN](https://arxiv.org/abs/1511.06581) - Separate value and advantage
- [Prioritized Experience Replay](https://arxiv.org/abs/1511.05952) - Sample important transitions more

---

## 🧹 Cleanup

In [ ]:
# Clean up
env.close()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

import gc
gc.collect()

print("✅ Notebook complete! Ready for Lab D.4: Policy Gradients and PPO")